In [10]:
import pandas as pd
amazon=pd.read_csv('Amazon_Reviews.csv')
amazon

,Review,Label
0,Stuning even for the non-gamer: This sound tr...,__label__2
1,The best soundtrack ever to anything.: I'm re...,__label__2
2,Amazing!: This soundtrack is my favorite musi...,__label__2
3,Excellent Soundtrack: I truly like this sound...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2
...,...,...
194,A Book That Is Worth a Second Look: This book...,__label__2
195,Best game ever: This games makes even amazing...,__label__2
196,Guitar in Absentia: With all due respect to a...,__label__1
197,Stiff and Smells like drying paint: You get w...,__label__1


In [11]:
from nltk.corpus import stopwords

#stopwords.words('English')

In [12]:
amazon['Label']=amazon['Label'].map({'__label__2 ':1,'__label__1 ':0})
y= amazon['Label']
amazon.drop(columns='Label',axis=1,inplace=True)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(amazon,y,test_size=0.2,random_state=40)

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings 
warnings.filterwarnings('ignore')

tokenizer=RegexpTokenizer(r'\w+')
lemmatizer=WordNetLemmatizer()
stemmer=PorterStemmer()

def preprocessing(review):
    
    final_tokens=''
    tokens=tokenizer.tokenize(review)
    #print('Tokens:',tokens)
    pure_tokens=[token.lower() for token in tokens if token.lower() not in stopwords.words('english')]
    #print('pure_Tokens:',pure_tokens)
    stemmed_tokens=[stemmer.stem(pure_token) for pure_token in pure_tokens]
    
    final_tokens=final_tokens.join(stemmed_tokens)
    
    return final_tokens

#preprocessing('amazon')
x_train['Cleaned_text']=x_train['Review'].apply(preprocessing)
#x_train['Cleaned_text']

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(stop_words='english',use_idf=True)

vectorizer.fit(x_train['Cleaned_text'])

x_test['Cleaned_text']=x_test['Review'].apply(preprocessing)
x_train_TfIdf=vectorizer.transform(x_train['Cleaned_text'])

#vectorizer.vocabulary_
x_test_tfIdf=vectorizer.transform(x_test['Cleaned_text'])

In [16]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score, recall_score, precision_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
clf=MultinomialNB().fit(x_train_TfIdf.toarray(),y_train)
y_pred=clf.predict(x_test_tfIdf.toarray())

In [18]:
print(confusion_matrix(y_test,y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))


y_proba_pred=clf.predict_proba(x_test_tfIdf.toarray())[::,1]

auc = roc_auc_score(y_test,fpr,tpr,thresholds=roc_curve(y_test,y_predic)

plt.figure(figsize=(12,12))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve')

plt.legend(loc=7)
plt.show())

print(auc)

SyntaxError: invalid syntax (<ipython-input-18-fba4eed0a03b>, line 11)

In [ ]:
fpr,tpr,thresholds=roc_curve(y_test,y_proba_pred)

plt.plot(fpr,tpr,label="auc="+str(auc))              
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve')

plt.legend(loc=5)
plt.show()